# Testing New Tumor Simulation Features

This notebook tests the newly implemented features:
1. FA-dependent anisotropic diffusion
2. Volume-preserving mass effect formulation
3. Adaptive time stepping
4. Regional material properties
5. Ventricular compliance modeling
6. Sparse matrix caching

In [ ]:
import numpy as np
import time
import os

from pft_fem import (
    MNIAtlasLoader,
    MRISimulator,
    TumorParameters,
    SolverConfig,
    MRISequence,
    save_nifti,
    get_regional_properties,
    BrainTissue,
    REGIONAL_STIFFNESS,
)

## 1. Test Regional Material Properties

In [ ]:
print("Regional Material Properties (Literature-based)")
print("=" * 50)

# Test different anatomical regions
regions = [
    ("Cerebellum", np.array([0.0, -60.0, -30.0])),
    ("Brainstem", np.array([0.0, -35.0, -45.0])),
    ("Fourth Ventricle", np.array([0.0, -45.0, -35.0])),
]

for name, coords in regions:
    props_gm = get_regional_properties(coords, BrainTissue.GRAY_MATTER)
    props_wm = get_regional_properties(coords, BrainTissue.WHITE_MATTER)
    props_csf = get_regional_properties(coords, BrainTissue.CSF)
    
    print(f"\n{name} (MNI: {coords}):")
    print(f"  GM:  E={props_gm.young_modulus:>6.0f} Pa, nu={props_gm.poisson_ratio:.2f}")
    print(f"  WM:  E={props_wm.young_modulus:>6.0f} Pa, nu={props_wm.poisson_ratio:.2f}")
    print(f"  CSF: E={props_csf.young_modulus:>6.0f} Pa, nu={props_csf.poisson_ratio:.3f}")

## 2. Load MNI Atlas

In [ ]:
print("Loading MNI Atlas...")
start = time.time()

loader = MNIAtlasLoader()
atlas = loader.load()

print(f"Loaded in {time.time() - start:.1f}s")
print(f"Shape: {atlas.shape}")
print(f"Voxel size: {atlas.voxel_size}")

## 3. Configure Tumor Parameters with New Features

In [ ]:
tumor_params = TumorParameters(
    # Standard parameters
    initial_radius=3.0,
    initial_density=0.9,
    proliferation_rate=0.05,
    diffusion_rate=0.01,
    
    # NEW: FA-dependent anisotropic diffusion
    # k=6 means at FA=0.5, D_parallel/D_perp = 4x
    fa_anisotropy_factor=6.0,
    
    # NEW: Physics-based volume-preserving mass effect
    use_volume_preserving_mass_effect=True,
    pressure_decay_length_factor=2.0,
    
    # NEW: Adaptive time stepping for efficiency
    use_adaptive_stepping=True,
    dt_min=1.0,
    dt_max=5.0,
    
    # NEW: Ventricular compliance (CSF is soft)
    use_ventricular_compliance=True,
    ventricular_compliance_factor=0.01,
)

print("Tumor Parameters:")
print(f"  Center (MNI): {tumor_params.center}")
print(f"  Initial radius: {tumor_params.initial_radius} mm")
print(f"  Proliferation rate: {tumor_params.proliferation_rate} /day")
print(f"  Diffusion rate: {tumor_params.diffusion_rate} mm²/day")
print(f"\nNew Features:")
print(f"  FA anisotropy factor: {tumor_params.fa_anisotropy_factor}")
print(f"  Volume-preserving mass effect: {tumor_params.use_volume_preserving_mass_effect}")
print(f"  Adaptive stepping: {tumor_params.use_adaptive_stepping}")
print(f"  Ventricular compliance: {tumor_params.use_ventricular_compliance}")

## 4. Create Simulator

In [ ]:
print("Creating MRI Simulator...")
start = time.time()

simulator = MRISimulator(
    atlas,
    tumor_params=tumor_params,
    solver_config=SolverConfig.default(),
    use_default_solver=True,
)

print(f"Created in {time.time() - start:.1f}s")

## 5. Run Simulation

In [ ]:
print("Running 30-day tumor growth simulation...")
print("(This may take a few minutes)\n")

start = time.time()

result = simulator.run_full_pipeline(
    duration_days=30,
    sequences=[MRISequence.T1, MRISequence.T2],
    verbose=True,
)

elapsed = time.time() - start
print(f"\nSimulation completed in {elapsed:.1f}s ({elapsed/60:.1f} min)")

## 6. Analyze Results

In [ ]:
states = result.tumor_states
initial_state = states[0]
final_state = states[-1]

print("Simulation Results")
print("=" * 50)

# Time stepping analysis
print(f"\nTime Stepping:")
print(f"  Total steps: {len(states) - 1}")
print(f"  Final time: {final_state.time:.1f} days")

if len(states) > 2:
    times = [s.time for s in states]
    dts = np.diff(times)
    print(f"  dt range: [{min(dts):.2f}, {max(dts):.2f}] days")
    print(f"  Mean dt: {np.mean(dts):.2f} days")

# Tumor growth
print(f"\nTumor Growth:")
print(f"  Initial volume: {initial_state.current_volume:.1f} mm³")
print(f"  Final volume: {final_state.current_volume:.1f} mm³")
if initial_state.current_volume > 0:
    print(f"  Growth factor: {final_state.current_volume / initial_state.current_volume:.2f}x")

# Tissue displacement
print(f"\nTissue Displacement:")
max_disp = np.max(np.linalg.norm(final_state.displacement, axis=1))
mean_disp = np.mean(np.linalg.norm(final_state.displacement, axis=1))
print(f"  Max displacement: {max_disp:.2f} mm")
print(f"  Mean displacement: {mean_disp:.4f} mm")

# Ventricular compliance
print(f"\nVentricular Compliance:")
print(f"  Initial ventricular vol: {initial_state.initial_ventricular_volume:.1f} mm³")
print(f"  Final ventricular vol: {final_state.ventricular_volume:.1f} mm³")

# Generated images
print(f"\nGenerated Images:")
for name, img in result.mri_images.items():
    print(f"  {name}: shape={img.shape}, range=[{img.min():.2f}, {img.max():.2f}]")

print(f"\nTumor mask: {np.sum(result.tumor_mask)} voxels")

## 7. Save Results

In [ ]:
output_dir = "simulation_output"
os.makedirs(output_dir, exist_ok=True)

# Save MRI images
for name, img in result.mri_images.items():
    path = f"{output_dir}/new_features_{name}.nii.gz"
    save_nifti(img, atlas.affine, path)
    print(f"Saved {path}")

# Save tumor mask
mask_path = f"{output_dir}/new_features_tumor_mask.nii.gz"
save_nifti(result.tumor_mask.astype(np.float32), atlas.affine, mask_path)
print(f"Saved {mask_path}")

# Save deformed atlas
deformed_path = f"{output_dir}/new_features_deformed.nii.gz"
save_nifti(result.deformed_atlas, atlas.affine, deformed_path)
print(f"Saved {deformed_path}")

print(f"\nAll results saved to {output_dir}/")

## 8. Visualize (Optional)

In [ ]:
# Optional: Simple visualization using matplotlib
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Get center slices
    t1_img = result.mri_images['T1']
    tumor_mask = result.tumor_mask
    
    center = [s//2 for s in t1_img.shape]
    
    # Axial view
    axes[0, 0].imshow(t1_img[:, :, center[2]].T, cmap='gray', origin='lower')
    axes[0, 0].set_title('T1 - Axial')
    
    axes[1, 0].imshow(tumor_mask[:, :, center[2]].T, cmap='hot', origin='lower')
    axes[1, 0].set_title('Tumor Mask - Axial')
    
    # Coronal view
    axes[0, 1].imshow(t1_img[:, center[1], :].T, cmap='gray', origin='lower')
    axes[0, 1].set_title('T1 - Coronal')
    
    axes[1, 1].imshow(tumor_mask[:, center[1], :].T, cmap='hot', origin='lower')
    axes[1, 1].set_title('Tumor Mask - Coronal')
    
    # Sagittal view
    axes[0, 2].imshow(t1_img[center[0], :, :].T, cmap='gray', origin='lower')
    axes[0, 2].set_title('T1 - Sagittal')
    
    axes[1, 2].imshow(tumor_mask[center[0], :, :].T, cmap='hot', origin='lower')
    axes[1, 2].set_title('Tumor Mask - Sagittal')
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/visualization.png", dpi=150)
    plt.show()
    
except ImportError:
    print("matplotlib not available for visualization")

## Summary

All new features have been tested:

1. **FA-dependent anisotropic diffusion** - Tumor cells migrate preferentially along high-FA white matter tracts
2. **Volume-preserving mass effect** - Physics-based formulation using volumetric strain
3. **Adaptive time stepping** - Efficient simulation with variable step sizes
4. **Regional material properties** - Literature-based values by anatomical region
5. **Ventricular compliance** - CSF regions compress before parenchyma
6. **Sparse matrix caching** - Faster diffusion solves